Este cuaderno es simplemente una versión anotada en Jupyter del programa CSDALindenberg.jl. 

Usa el operador convexo de Lindenberg como Laplaciano.
Para usarse requiere un archivo *jld que contenga el LFP
como fue registrado en el experimento (o promediado sobre eventos evocados),
y, optativamente, una lista de electrodos inusables ("saturados").

In [2]:
push!(LOAD_PATH, ".")

4-element Array{String,1}:
 "@"      
 "@v#.#"  
 "@stdlib"
 "."      

In [3]:
using HDF5
using LindenbergOperadores, ParaSets
#importall LindenbergOperadores

In [33]:
arxnombre="../BRWFiles/test.h5"
arx=h5open(arxnombre)
names(arx)

8-element Array{String,1}:
 "CSDALindenberg"  
 "CanalesSaturados"
 "Canalesrespuesta"
 "LFPSaturados"    
 "final"           
 "freq"            
 "latencia"        
 "retrazo"         

In [23]:
# cargando los datos importantes
if in("LFPSaturados", elementos)
    LFP=read(arx["LFPSaturados"])
else
    LFP=read(arx["LFPTotal"])
end
saturadosarray=read(arx["CanalesSaturados"])
respuestasarray=read(arx["Canalesrespuesta"]);
frecuencia=read(arx["freq"])
close(arx)

In [24]:
saturados=arraytoset(saturadosarray)
respuestas=arraytoset(respuestasarray)

Set(Any[[25, 55], [22, 48], [19, 59], [48, 35], [20, 52], [17, 50], [25, 51], [23, 58], [21, 54], [20, 48]  …  [20, 54], [31, 53], [22, 52], [27, 52], [18, 57], [25, 52], [19, 57], [25, 59], [23, 51], [20, 50], [20, 56]])

In [25]:


lfpParchado=copy(LFP)
(l,m,cmax)=size(lfpParchado)
#Poner en cero los canales inservibles
for m in saturados
    q=m[1]
    p=m[2]
    lfpParchado[q,p,:]=zeros(cmax)
end

In [26]:
listaredux=TiraOrillas(saturados)


Set(Any[[29, 58], [34, 7], [29, 55], [33, 60], [38, 61], [32, 57], [27, 61], [25, 51], [33, 61], [36, 63]  …  [46, 57], [37, 60], [29, 40], [29, 5], [35, 49], [36, 57], [31, 61], [34, 56], [23, 52], [33, 59], [36, 59]])

In [27]:
#Creamos una mancha suave sobre el canal saturado.
for m in listaredux
        q=m[1]
        p=m[2]
        vecinos=vecindad8(m)
        lfpParchado[q,p,:]=promediasobreconjunto(vecinos,lfpParchado)
end


In [28]:
size(lfpParchado)

(64, 64, 459)

In [29]:
#en realidad nunca haces referencia al tercer numero como tiempo
(mu,nu,lu)=size(lfpParchado)
#Aplicamos un suavizado Gaussiano Temporal (esto afecta mucho las animaciones)
lfpplanchado=zeros(mu,nu,lu)
for j=1:mu,l=1:nu
    porromponpon=vec(lfpParchado[j,l,:])
    lfpplanchado[j,l,:]=GaussSuavizarTemporal(porromponpon)
end



In [14]:

aux1=zeros(mu,nu,lu)
aux2=zeros(mu,nu,lu)
#Suavizamos espacialmente con un filtro Gaussiano bidimensional el LFP.
#Posteriormente sacamos el dCSD.
for t=1:lu
    aux1[:,:,t]=GaussianSmooth(lfpplanchado[:,:,t])
    aux2[:,:,t]=DiscreteLaplacian(aux1[:,:,t])
end
CSD=-aux2;


In [30]:

h5open(arxnombre, "r+") do file
    write(file, "CSDALindenberg", CSD)  # alternatively, say "@write file A"
end


In [31]:
arx=h5open(arxnombre)
names(arx)

8-element Array{String,1}:
 "CSDALindenberg"  
 "CanalesSaturados"
 "Canalesrespuesta"
 "LFPSaturados"    
 "final"           
 "freq"            
 "latencia"        
 "retrazo"         

In [32]:
close(arx)